训练贝叶斯分类器模型
在计算后验概率时，我们需要计算出：

$$p(Y\big|X)=\frac{p(X\big|Y)p(Y)}{p(X)}$$

其中$$p(Y)$$为先验概率，先验信息不需要任何有关样本特征的信息，只需统计Y发生概率的初始估计

其中$$p(X)$$证据信息是X发生的概率，是所有类别的联合分布边缘化

其中似然 $$p(X|Y)$$ 是在给定类别 Y 下，观察到数据 X 的概率。它描述了特征在给定类别下的分布情况。

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    [ 'Sunny', 'Hot', 'High', 'Strong', 'No'],
    [ 'Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    [ 'Rain', 'Mild', 'High', 'Weak', 'Yes'],
    [ 'Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    [ 'Rain', 'Cool', 'Normal', 'Strong', 'No'],
    [ 'Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    [ 'Sunny', 'Mild', 'High', 'Weak', 'No'],
    [ 'Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    [ 'Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    [ 'Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    [ 'Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    [ 'Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    [ 'Rain', 'Mild', 'High', 'Strong', 'No'],
]

In [3]:
Data = pd.DataFrame(data, columns=[ 'Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis'])
Data.head()

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [4]:
X_train=Data[['Outlook','Temperature','Humidity','Wind']]
Y_train=Data[['PlayTennis']]
featureNames=X_train.columns

In [5]:
def Naive_Bayes(X_data,Y_data):
    y=Y_train.values
    X=X_train.values
    y_unique=np.unique(y)
    prior_prob=np.zeros(len(y_unique))
        
    for i in range(len(y_unique)):
        prior_prob[i]=sum(y==y_unique[i])/len(y)
            
    condition_prob={}
        
    for feat in featureNames:
        x_unique=list(set(X_data[feat]))
        x_condition_prob=np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j,k]=\
                sum((X_data[feat]==x_unique[k])&(Y_data.PlayTennis==y_unique[j]))/sum(y==y_unique[j])
        x_condition_prob=pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat]=x_condition_prob
    return prior_prob,condition_prob
    
    
    
def Prediction(testData,prior,condition_prob):
    numclass=prior.shape[0]
    numsample=testData.shape[0]
    featureNames=testData.columns
    post_prob=np.zeros((numsample,numclass))
    for k in range(numsample):
        prob_k=np.zeros((numclass,))
        for i in range(numclass):
            if(i==0):
                a='No'
            if(i==1):
                a='Yes'
            pri=prior[i]
            for feat in featureNames:
                feat_val=testData[feat][k]
                cp=condition_prob[feat]
                cp_val=cp.loc[a,feat_val]
                pri*=cp_val
            prob_k[i]=pri
        prob=prob_k/np.sum(prob_k,axis=0)
        post_prob[k,:]=prob
    return post_prob
    
    

In [6]:
test_data=[['Sunny','Cool','High','Strong']]
testData=pd.DataFrame(test_data,columns=[ 'Outlook', 'Temperature', 'Humidity', 'Wind'])

In [7]:
prior_prob,condition_prob=Naive_Bayes(X_train,Y_train)

print(prior_prob)
print(condition_prob[ 'Outlook'])
print(condition_prob[ 'Temperature'])
print(condition_prob[ 'Humidity'])
print(condition_prob[ 'Wind'])
postPrior=Prediction(testData,prior_prob,condition_prob)
postPrior

[0.35714286 0.64285714]
         Rain     Sunny  Overcast
No   0.400000  0.600000  0.000000
Yes  0.333333  0.222222  0.444444
         Cool       Hot      Mild
No   0.200000  0.400000  0.400000
Yes  0.333333  0.222222  0.444444
       Normal      High
No   0.200000  0.800000
Yes  0.666667  0.333333
       Strong      Weak
No   0.600000  0.400000
Yes  0.333333  0.666667


array([[0.79541735, 0.20458265]])

In [8]:
if postPrior[0][0]>postPrior[0][1]:
    print("No")
else:
    print("Yes")

No
